In [1]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

In [7]:
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [2]:
df = pd.read_csv("../data/export_device_1_basic_aggregated_15minutes.csv")
df.head(10)

,Device ID,Start Time,End Time,Record Count,Grid Feed In,External Energy Supply,Internal Power Supply,Current Power,Self Consumption,Ambient Temperature,Module Temperature,Total Irradiation
0,1,2021-11-06 11:00:00,2021-11-06 11:15:00,30,9080.8,0.0,37015.066667,46095.866667,37015.066667,30.046667,57.706667,79.340000
1,1,2021-11-06 11:15:00,2021-11-06 11:30:00,120,9427.4,1.6,37756.825000,47184.225000,37756.825000,30.140833,58.147500,80.698333
2,1,2021-11-06 11:30:00,2021-11-06 11:45:00,120,11218.2,258.0,36041.425000,47259.625000,36041.425000,30.385833,57.361667,81.116667
3,1,2021-11-06 11:45:00,2021-11-06 12:00:00,120,14011.4,0.0,33460.216667,47471.616667,33460.216667,31.025833,57.097500,80.676667
4,1,2021-11-06 12:00:00,2021-11-06 12:15:00,120,11030.2,0.0,38093.108333,49123.308333,38093.108333,31.340000,56.047500,81.843333
5,1,2021-11-06 12:15:00,2021-11-06 12:30:00,120,17090.6,0.0,33160.058333,50250.658333,33160.058333,32.125833,56.274167,82.270000
6,1,2021-11-06 12:30:00,2021-11-06 12:45:00,120,23212.8,0.0,26659.975000,49872.775000,26659.975000,32.125833,56.409167,81.435833
7,1,2021-11-06 12:45:00,2021-11-06 13:00:00,120,25765.6,0.0,23761.108333,49526.708333,23761.108333,31.910833,55.769167,80.035833
8,1,2021-11-06 13:00:00,2021-11-06 13:15:00,120,24789.2,0.0,25968.075000,50757.275000,25968.075000,32.567500,55.317500,77.674167
9,1,2021-11-06 13:15:00,2021-11-06 13:30:00,120,24477.8,100.6,26190.091667,50667.891667,26190.091667,32.662500,54.140000,76.503333


In [ ]:

# Assuming 'df' is your DataFrame and it has a 'Start Time' column
# First, let's ensure the 'Start Time' column is in datetime format
date_col = 'Start Time'
df[date_col] = pd.to_datetime(df[date_col])
print(df[date_col].dt.year.unique())
print(df[date_col].dt.year.value_counts())

[2021 2022 2023 2024 2025]
2023    31394
2022    25611
2024    12600
2025     4836
2021     3972
Name: Start Time, dtype: int64


In [ ]:
# Sort the DataFrame by 'Start Time'
df = df.sort_values('Start Time')

# Define the start and end times
start_time = df['Start Time'].min()
end_time = df['Start Time'].max()

# Create a complete time series with 15-minute intervals
complete_time_series = pd.date_range(start=start_time, end=end_time, freq='15min')

# Create a new DataFrame with the complete time series
complete_df = pd.DataFrame({'Start Time': complete_time_series})

# Merge the original data with the complete time series
merged_df = pd.merge(complete_df, df, on='Start Time', how='left')

# Fill missing values with 0
merged_df = merged_df.fillna(0)

# Now 'merged_df' contains the complete time series with no missing intervals
# Replace the original DataFrame if needed
df = merged_df